# 📝 Decorators and Abstraction in Python Classes 🎨📚

![](./assets/figures/decorating_gif.gif)


## Decorators in Classes

- Definition: A decorator in Python is a function that takes another function (or method) and extends its behavior.
- Common Class Decorators:

1. `@property` – Converts a method into a getter for an attribute.

2. `@<property>.setter` – Adds a setter method to the property.

3. `@classmethod` – Method belongs to the class, not the instance. Receives `cls` as the first argument.

4. `@staticmethod` – Method belongs to the class’s namespace but receives no automatic arguments.

Why Decorators?
They help create cleaner APIs, reduce boilerplate code, and organize logic (e.g., data validation, class-wide utilities) in a consistent way.


##  `@property` and `@<property>.setter` in Detail


### Simple Example with Validation


In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self._width = width
        self._height = height

    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        if value < 0:
            raise ValueError("Width must be non-negative.")
        self._width = value

    @property
    def height(self):
        return self._height

    @height.setter
    def height(self, value):
        if value < 0:
            raise ValueError("Height must be non-negative.")
        self._height = value

    @property
    def area(self):
        """Read-only property (no setter)"""
        return self._width * self._height


# Usage
rect = Rectangle(10, 5)
print(rect.area)  # 50
rect.width = 20  # Valid
rect.height = -10  # Raises ValueError

1. `area` is a read-only property (no setter) – it’s computed from `_width` and `_height`.

2. Both `width` and `height` have validation logic in their setters.


## More Examples: `@property` with Computed Attributes


### Example: Bounding Box with Automatic Width/Height Updates


In [ ]:
class BoundingBox:
    def __init__(self, x1, y1, x2, y2):
        self._x1 = x1
        self._y1 = y1
        self._x2 = x2
        self._y2 = y2

    @property
    def width(self):
        return abs(self._x2 - self._x1)

    @property
    def height(self):
        return abs(self._y2 - self._y1)

    @property
    def x1(self):
        return self._x1

    @x1.setter
    def x1(self, val):
        self._x1 = val
        # Could trigger logs, or re-calculate something if needed

    # Similarly, x2, y1, y2 properties if needed


bb = BoundingBox(0, 0, 10, 5)
print(bb.width, bb.height)  # 10, 5
bb.x1 = 2
print(bb.width, bb.height)  # 8, 5 (auto-updated)


- Whenever `x1` changes, `width` effectively updates because it’s computed from `_x1` and `_x2`.
- This approach keeps logic consistent and user-friendly.


## `@classmethod` and `@staticmethod`


### `@classmethod`

A class method is a method that is bound to the class and not the instance of the class. They have access to the class state that applies across all instances of the class. Class methods are marked with the `@classmethod` decorator and take `cls` as the first parameter, which refers to the class itself.

Class methods are useful when you need to perform operations that pertain to the class as a whole, rather than to any particular instance. They are often used for factory methods that instantiate an instance of the class using alternative constructors, or for methods that need to modify class-level attributes.

- Receives `cls` as the first parameter.
- Often used as alternative constructors or class-level factories.


In [ ]:
class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @classmethod
    def from_tuple(cls, coord_tuple):
        return cls(coord_tuple[0], coord_tuple[1])


# Usage
v = Vector2D.from_tuple((3, 4))
print(v.x, v.y)  # 3, 4

### `@staticmethod`

Static methods are used when you need a function that logically belongs to a class but does not require access to any instance or class-specific data. They help in organizing code within the class's namespace, making it clear that the function is related to the class, even though it doesn't interact with the class or its instances.

- Does not receive `self` or `cls`.
- Functions that conceptually belong to a class’s domain but do not need instance or class data.


In [ ]:
class VectorUtilities:
    @staticmethod
    def dot_product(v1, v2):
        return v1.x * v2.x + v1.y * v2.y


# Usage
result = VectorUtilities.dot_product(Vector2D(1, 2), Vector2D(3, 4))
print(result)  # 11

## Abstract Classes for Enforced Interfaces

- Abstraction: Defines a contract that child classes must fulfill.
- `abc` Module: Use `ABC` (Abstract Base Class) and `@abstractmethod`.


In [ ]:
from abc import ABC, abstractmethod


class Sensor(ABC):
    @abstractmethod
    def read_data(self):
        pass

    @property
    @abstractmethod
    def unit(self):
        """Read-only property specifying the unit of measurement"""
        pass


- Any concrete subclass must implement `read_data` and `unit`.


### Concrete Subclass Example


In [ ]:
class TemperatureSensor(Sensor):
    def __init__(self, location):
        self._location = location
        self._last_reading = 0.0

    def read_data(self):
        # Simulate reading from a real sensor
        self._last_reading += 1.5
        return self._last_reading

    @property
    def unit(self):
        return "Celsius"


# Usage
sensor = TemperatureSensor("Kitchen")
sensor.read_data()
sensor.unit

## Additional Examples: Complex Validation + Abstract Patterns


### Example: Calibrated Sensor with Setters


In [ ]:
class CalibratedSensor(Sensor):
    def __init__(self, offset=0):
        self._offset = offset
        self._latest = 0.0

    def read_data(self):
        # Hypothetical raw sensor value
        raw_value = 42.0
        self._latest = raw_value + self._offset
        return self._latest

    @property
    def offset(self):
        return self._offset

    @offset.setter
    def offset(self, val):
        # For example, offset must be within -10 to 10
        if not (-10 <= val <= 10):
            raise ValueError("Offset out of supported range.")
        self._offset = val

    @property
    def unit(self):
        return "units"


sensor = CalibratedSensor()
print(sensor.read_data())  # e.g. 42.0
sensor.offset = 5
print(sensor.read_data())  # e.g. 47.0
# sensor.offset = 15         # Raises ValueError


- The setter for `offset` ensures calibration remains within acceptable parameters.
- `read_data()` references `_offset` to produce a final reading.


## Best Practices & Takeaways


1. Use `@property` for Readable APIs
    - Attributes are accessed like `obj.attribute` without exposing raw internal state directly.

2. Validate with Setters
    - Keep constraints or checks in the property’s setter, preventing invalid state.

3. Class-Level Helpers
    - `@classmethod` for alternative constructors or class-level logic (parsing from strings, tuples, dicts, etc.).
    - `@staticmethod` for utility methods that do not rely on class or instance data.

4. Abstraction with `abc`
    - Define an abstract base class with `@abstractmethod` to enforce required methods/properties in child classes.
    - Ensures all subclasses share consistent method signatures.

5. Documentation
    - Docstrings for properties (`"""Getter: ..."""`, `"""Setter: ..."""`) clarify usage and constraints.

6. Simplicity & Clarity
    - Avoid overcomplicating property or abstract class logic. Keep it straightforward and aligned with your design goals.


## Summary

- Decorators (e.g., `@property`, `@classmethod`, `@staticmethod`) streamline class design and reduce boilerplate.
- Setters & Getters with properties provide a *Pythonic* way to combine data encapsulation and user-friendly attribute access.
- Abstract Base Classes (`ABC`) allow you to define a contract that all derived classes must fulfill, ensuring consistent interfaces across your codebase.

By combining these tools, you can build clean, maintainable, and flexible class hierarchies in Python, accommodating both simple use cases (like basic validation) and advanced frameworks (like robust sensor systems or plugin architectures).
